In [ ]:
# Función para generar la ruta de S3 a partir de la fecha y el productodef build_goes17_s3_path(product, satellite, date):     year = date.strftime("%Y")     day_of_year = date.strftime("%j")     hour = date.strftime("%H") file_name = f"OR_{product}-M6C02_G{satellite}_s{year}{day_of_year}{hour}0000.nc" s3_path = f"s3://noaa-goes17/{product}/{year}/{day_of_year}/{hour}/{file_name}"return s3_path

In [6]:
import xarray as xr
import s3fs
from datetime import datetime

# Función para generar la ruta de S3 a partir de la fecha y el producto
def build_goes17_s3_path(product, satellite, date):
    year = date.strftime("%Y")
    day_of_year = date.strftime("%j")
    hour = date.strftime("%H")
    
    file_name = f"OR_{product}-M6C02_G{satellite}_s{year}{day_of_year}{hour}0000.nc"
    s3_path = f"s3://noaa-goes17/{product}/{year}/{day_of_year}/{hour}/{file_name}"
    
    return s3_path

# Función para recortar los datos de GOES-17 según los límites geográficos
def crop_goes17_data(file_path, lat_bnds, lon_bnds):
    # Abrir el archivo desde S3 usando s3fs
    fs = s3fs.S3FileSystem(anon=True)
    ds = xr.open_dataset(fs.open(file_path), engine="h5netcdf")
    
    # Definir los límites geográficos para el recorte
    ds_cropped = ds.sel(lat=slice(lat_bnds[0], lat_bnds[1]), lon=slice(lon_bnds[0], lon_bnds[1]))
    
    return ds_cropped

# Función principal para descargar los recortes de GOES-17
def download_goes17_crops(product, satellite, dates, lat_bnds, lon_bnds):
    for date in dates:
        # Generar la ruta del archivo en S3
        s3_path = build_goes17_s3_path(product, satellite, date)
        print(f"Procesando archivo: {s3_path}")
        
        try:
            # Recortar los datos de la región de interés
            ds_cropped = crop_goes17_data(s3_path, lat_bnds, lon_bnds)
            
            # Guardar el archivo recortado localmente
            local_file = f"GOES17_crop_{date.strftime('%Y%m%d%H')}.nc"
            ds_cropped.to_netcdf(local_file)
            print(f"Archivo recortado guardado como: {local_file}")
        
        except Exception as e:
            print(f"Error al procesar {s3_path}: {e}")

# Ejemplo de uso
if __name__ == "__main__":
    # Lista de fechas (puedes agregar más fechas)
    dates = [
        # datetime(2023, 9, 1, 0, 0),  # Formato: año, mes, día, hora, minuto
        # datetime(2024, 9, 1, 3, 0),
        datetime(2023, 9, 1, 6, 0)
    ]
    
    # Producto GOES-17 y satélite
    product = "ABI-L2-CMIPF"
    satellite = "16"
    
    # Definir los límites geográficos para Colombia (puedes ajustarlos)
    x_lon = 10
    x_lat = 10
    extent = [-73.722451-x_lon, 9.789103-x_lat, -73.722451+x_lon, 9.789103+x_lat]
    lat_bnds, lon_bnds = [extent[1], extent[3]], [extent[0], extent[2]]

    # lat_bnds = [min_lat, max_lat]  # Por ejemplo: [min_lat, max_lat] = [12.0, 4.0]
    # lon_bnds = [min_lon, max_lon]  # Por ejemplo: [min_lon, max_lon] = [-80.0, -66.0]
    
    # Descargar y recortar los archivos
    download_goes17_crops(product, satellite, dates, lat_bnds, lon_bnds)


Procesando archivo: s3://noaa-goes17/ABI-L2-CMIPF/2023/244/06/OR_ABI-L2-CMIPF-M6C02_G16_s2023244060000.nc
Error al procesar s3://noaa-goes17/ABI-L2-CMIPF/2023/244/06/OR_ABI-L2-CMIPF-M6C02_G16_s2023244060000.nc: noaa-goes17/ABI-L2-CMIPF/2023/244/06/OR_ABI-L2-CMIPF-M6C02_G16_s2023244060000.nc


In [10]:
date = datetime(2023, 9, 1, 6, 0)
satellite = "17"
product = "ABI-L2-CMIPF"
s3_path = build_goes17_s3_path(product, satellite, date)
s3_path

's3://noaa-goes17/ABI-L2-CMIPF/2023/244/06/OR_ABI-L2-CMIPF-M6C02_G17_s2023244060000.nc'

In [12]:
import datetime
import os
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import xarray as xr
import s3fs
from dask import delayed
# from datetime import datetime

# path_dest = 'F:/GOES_Files/'
path_dest =  'D:/RODRIGO/IntradayForecasting/content/GOES_Files/CMIPF_sliced_resized/'
date_save = '20240904'
time_save = '1520'
band = 5
product_name = 'ABI-L2-CMIPF'
bucket_name = 'noaa-goes16'

def download_goes_slcrz(path_dest, date_save, time_save, product_name, band, bucket_name ='noaa-goes16', target_x_res = 300, target_y_res = 300):

    yyyymmddhhmn = date_save + time_save
    os.makedirs(path_dest, exist_ok=True)

    year = datetime.datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
    day_of_year = datetime.datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%j')
    hour = datetime.datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
    min = datetime.datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')

    # AMAZON repository information
    # https://noaa-goes16.s3.amazonaws.com/index.html

    #product_name = 'ABI-L2-CMIPF'

    # Initializes the S3 client
    s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    #-----------------------------------------------------------------------------------------------------------
    # File structure
    prefix = f'{product_name}/{year}/{day_of_year}/{hour}/OR_{product_name}-M6C{int(band):02.0f}_G16_s{year}{day_of_year}{hour}{min}'

    # Seach for the file on the server
    s3_result = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter = "/")

    for obj in s3_result['Contents']:
        key = obj['Key']
        file_name = key.split('/')[-1].split('.')[0]
    if os.path.exists(f'{path_dest}/{file_name}.nc'):
        print(f'File {path_dest}/{file_name}.nc exists')
    else:
        file_path = f's3://{bucket_name}/{key}'

        # Abrir el archivo desde S3 usando s3fs
        fs = s3fs.S3FileSystem(anon=True)
        ds = xr.open_dataset(fs.open(file_path), engine="h5netcdf")

        # Definir los límites geográficos para el recorte
        ds_sel = delayed(ds.sel)(x=slice(-0.05, 0.07), y=slice(0.09, -0.03))
        original_x_size = delayed(ds_sel.dims['x'])
        original_y_size = delayed(ds_sel.dims['y'])
        scale_factor_x = delayed(lambda ox: max(ox // target_x_res, 1))(original_x_size)
        scale_factor_y = delayed(lambda oy: max(oy // target_y_res, 1))(original_y_size)

        # Apply the downscaling factor to each image
        ds_downscaled = delayed(ds_sel.coarsen)(x=scale_factor_x, y=scale_factor_y, boundary="trim").mean()

        print(f'Downloading file {path_dest}/{file_name}.nc')
        output_path = f'{path_dest}/{file_name}.nc'
        downscaled = ds_downscaled.compute()
        downscaled.to_netcdf(output_path)

In [13]:
path_dest =  'D:/RODRIGO/IntradayForecasting/content/GOES_Files/CMIPF_sliced_resized/'
date_save = '20240904'
time_save = '1520'
band = 5
product_name = 'ABI-L2-CMIPF'
# bucket_name = 'noaa-goes16'

download_goes_slcrz(path_dest, date_save, time_save, product_name, band)

d:\RODRIGO\IntradayForecasting\Condashboard\Lib\site-packages\dask\core.py:127: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  return func(*(_execute_task(a, cache) for a in args))


C:\Users\rodhe\AppData\Local\Temp\ipykernel_6832\3964174017.py:8: SerializationWarning: saving variable y with floating point data as an integer dtype without any _FillValue to use for NaNs
  download_goes_slcrz(path_dest, date_save, time_save, product_name, band)
C:\Users\rodhe\AppData\Local\Temp\ipykernel_6832\3964174017.py:8: SerializationWarning: saving variable x with floating point data as an integer dtype without any _FillValue to use for NaNs
  download_goes_slcrz(path_dest, date_save, time_save, product_name, band)
